## Geoserver rest API
* [API docs](https://docs.geoserver.org/latest/en/user/rest/index.html#rest) BROKEN
* [API legacy](https://docs.geoserver.org/2.13.2/user/rest/api/index.html#rest-api)
* XML :-/
* Useful link: curl to requests at https://curl.trillworks.com/

In [15]:
!curl -u admin:geoserver -XGET -H "Accept: text/xml" http://localhost:8080/geoserver/rest/about/version.xml

<about>
  <resource name="GeoServer">
    <Build-Timestamp>21-Sep-2018 20:18</Build-Timestamp>
    <Version>2.14.0</Version>
    <Git-Revision>2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c</Git-Revision>
  </resource>
  <resource name="GeoTools">
    <Build-Timestamp>21-Sep-2018 16:31</Build-Timestamp>
    <Version>20.0</Version>
    <Git-Revision>ffc271f317c04e714ea44a4879dd4601bd723d5e</Git-Revision>
  </resource>
  <resource name="GeoWebCache">
    <Version>1.14.0</Version>
    <Git-Revision>1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317</Git-Revision>
  </resource>
</about>

In [16]:
import requests

headers = {
    'Accept': 'text/xml',
}


response = requests.get('http://localhost:8080/geoserver/rest/about/version.xml', headers=headers, auth=('admin', 'geoserver'))

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('http://localhost:8080/geoserver/rest/about/manifest.xml?key=GeoServerModule', headers=headers, auth=('admin', 'geoserver'))


In [17]:
response.text

'<about>\n  <resource name="GeoServer">\n    <Build-Timestamp>21-Sep-2018 20:18</Build-Timestamp>\n    <Version>2.14.0</Version>\n    <Git-Revision>2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c</Git-Revision>\n  </resource>\n  <resource name="GeoTools">\n    <Build-Timestamp>21-Sep-2018 16:31</Build-Timestamp>\n    <Version>20.0</Version>\n    <Git-Revision>ffc271f317c04e714ea44a4879dd4601bd723d5e</Git-Revision>\n  </resource>\n  <resource name="GeoWebCache">\n    <Version>1.14.0</Version>\n    <Git-Revision>1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317</Git-Revision>\n  </resource>\n</about>'

Lets try [requests-xml](https://github.com/erinxocon/requests-xml/) installed with `requests-xml`, and we convert the response to json string and then to a python dict

In [4]:
import json
from requests_xml import XMLSession
session = XMLSession()

In [18]:
r = session.get('http://localhost:8080/geoserver/rest/about/version.xml', headers=headers, auth=('admin', 'geoserver'))

In [19]:
json_data = r.xml.json()

In [20]:
data_dict = json.loads(json_data)

In [22]:
data_dict

{'about': {'resource': [{'@name': 'GeoServer',
    'Build-Timestamp': {'$': '21-Sep-2018 20:18'},
    'Version': {'$': '2.14.0'},
    'Git-Revision': {'$': '2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c'}},
   {'@name': 'GeoTools',
    'Build-Timestamp': {'$': '21-Sep-2018 16:31'},
    'Version': {'$': 20.0},
    'Git-Revision': {'$': 'ffc271f317c04e714ea44a4879dd4601bd723d5e'}},
   {'@name': 'GeoWebCache',
    'Version': {'$': '1.14.0'},
    'Git-Revision': {'$': '1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317'}}]}}

In [23]:
data_dict['about']['resource']

[{'@name': 'GeoServer',
  'Build-Timestamp': {'$': '21-Sep-2018 20:18'},
  'Version': {'$': '2.14.0'},
  'Git-Revision': {'$': '2ecf6413b0d5d304c7dc23d8d7e9f3cfad29f37c'}},
 {'@name': 'GeoTools',
  'Build-Timestamp': {'$': '21-Sep-2018 16:31'},
  'Version': {'$': 20.0},
  'Git-Revision': {'$': 'ffc271f317c04e714ea44a4879dd4601bd723d5e'}},
 {'@name': 'GeoWebCache',
  'Version': {'$': '1.14.0'},
  'Git-Revision': {'$': '1.14.x/775e94eb2ae59dd9b74aecc709b67d7b968a4317'}}]

# Post to Geoserver
The [Geoserver Cookbook](https://books.google.co.uk/books?id=oMaiBQAAQBAJ&pg=PT427&lpg=PT427&dq=geoserver+curl+csv&source=bl&ots=gXfsIuzbWA&sig=QxUba_R1xo-sdH4tfHjSAPQdk0M&hl=en&sa=X&ved=2ahUKEwj5rvqL9I_eAhXnIcAKHZhoD6AQ6AEwCHoECCkQAQ#v=onepage&q=geoserver%20curl%20csv&f=false) will be useful.

Using requests files are posted like:
```python
url = 'https://httpbin.org/post'
files = {'file': open('report.xls', 'rb')}

r = requests.post(url, files=files)
>>> r.text
```